# Análise Exploratória de Dados: Metacritic - Melhores Jogos de Todos os Tempos
<div align="center">
    <img width="800" src="images/print_metacritic.jpg">
</div>

O Metacritic é conhecido por ser um site com uma crítica excelente aos jogos, filmes/séries, músicas e shows de TV.
Nele podemos ver reviews de revistas ou sites da crítica que compõe uma nota final conhecida como <em><strong>Metascore</strong></em>, e também podemos ver notas e reviews dos usuários que forma o <em><strong>Userscore</strong></em>, e estes determinam se tal filme, série ou jogo <em>"está bem na fita"</em> como esse do [GTA Trilogy](https://www.metacritic.com/game/pc/grand-theft-auto-the-trilogy---the-definitive-edition) que virou até [meme](https://gamerant.com/gta-trilogy-definitive-edition-grand-theft-auto-memes/) por está em uma das piores notas de todos os tempos para jogos. <br>

## Melhores jogos de todos os tempos

Em uma das [abas](https://www.metacritic.com/browse/games/score/metascore/all/all/filtered) do Metacritic há um ranking por <em><strong>Metascore</strong></em> de Jogos, e é onde estão os jogos de pelo menos 7 reviews dos críticos<sup><a href="#fonte1">1</a></sup> no site. E lá podemos observar os melhores, os mais clássicos, os mais atuais, e os piores. <br>
Para fim de projeto irei analisar esse ranking tirando proveito das ferramentas do Python: `Pandas` e `Plotly`

Ao examinar como o Metascore é feito, o metacritic divide o Metascore em 3 tipos<sup><a href="#fonte2">2</a></sup>:
* `Verde` - Bom
* `Amarelo` - Médio
* `Vermelho` - Ruim
    
Mas para ser mais preciso, é assim que os pontos são organizados:

<img width="600em" src="images/print_metascore.jpg">

# Let's Analyze

O que podemos tirar de proveito desse dataset? Elaborei algumas perguntas para nosso EDA(Exploratory Data Analysis)
[Link para o Download do Dataset](https://www.kaggle.com/caiquerezende/metacritics-best-video-games-of-all-time-2021)

Antes disso, vamos saber como nossos dados estão organizados

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# vamos usar o delimitador que está com ';' em vez de ','. e converter a coluna 'Release Date' de object para datetime64
metacritic = pd.read_csv('dataset/Best Video Games of All Time.csv', delimiter=';', parse_dates=["Release Date"])
metacritic.head()

,Game,Ranking,Metascore,Platform,Release Date
0,The Legend of Zelda: Ocarina of Time,1,99,Nintendo 64,1998-11-23
1,Tony Hawk's Pro Skater 2,2,98,PlayStation,2000-09-20
2,Grand Theft Auto IV,3,98,PlayStation 3,2008-04-29
3,SoulCalibur,4,98,Dreamcast,1999-09-08
4,Grand Theft Auto IV,5,98,Xbox 360,2008-04-29


In [3]:
metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19106 entries, 0 to 19105
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Game          19106 non-null  object        
 1   Ranking       19106 non-null  int64         
 2   Metascore     19106 non-null  int64         
 3   Platform      19106 non-null  object        
 4   Release Date  19106 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 597.1+ KB


Vemos que não há nenhum valor nulo em nosso dataset, e que foi convertido o `Release Date` para datetime64, e nosso dataset está pesando `~600KB` em nossa máquina. Agora podemos ver um resumo estatístico dos nossos dados numéricos, `Ranking` e `Metascore`

In [4]:
metacritic.describe().round(1)

,Ranking,Metascore
count,19106.0,19106.0
mean,9553.5,70.6
std,5515.6,12.3
min,1.0,11.0
25%,4777.2,64.0
50%,9553.5,72.0
75%,14329.8,79.0
max,19106.0,99.0


Podemos ver que para `Ranking` há medidas estranhas, pois são todos únicos. Podemos provar da seguinte forma.

In [5]:
# visualizar o número de linhas e colunas do dataset
metacritic.shape

(19106, 5)

In [6]:
metacritic["Ranking"].value_counts().sum() == len(metacritic.index)

True

Podemos observar que cada valor é único pois sua soma dá o tamanho das linhas dentro do dataset.

Podemos também subdividir os pontos(`scores`) em bins/buckets de tipos de metascores, como vimos lá em cima

In [7]:
types_of_metascore = ["Overwhelming Dislike", "Unfavorable Reviews", "Mixed or Average", "Favorable Reviews", "Universal Acclaim"]
bins = [0, 19, 49, 74, 89, 100]
metacritic["Type Of Metascore"] = pd.cut(metacritic["Metascore"], bins=bins, labels=types_of_metascore)

In [8]:
metacritic["Type Of Metascore"].value_counts()

Mixed or Average        9797
Favorable Reviews       7553
Unfavorable Reviews     1202
Universal Acclaim        545
Overwhelming Dislike       9
Name: Type Of Metascore, dtype: int64

## Vamos analisar mais a fundo

* Quantos jogos há por plataforma?

In [9]:
# se quisessemos usar o groupby
# metacritic.groupby("Platform").count()["Game"].sort_values(ascending=False)

# para simplificar
metacritic["Platform"].value_counts()

PC                  4968
PlayStation 4       2073
Xbox 360            1665
Switch              1456
PlayStation 2       1421
PlayStation 3       1265
Xbox One            1182
Xbox                 794
DS                   728
Wii                  663
PSP                  514
GameCube             452
Game Boy Advance     442
3DS                  398
PlayStation Vita     258
PlayStation          189
Wii U                186
PlayStation 5        153
Dreamcast            126
Xbox Series X         97
Nintendo 64           71
Stadia                 5
Name: Platform, dtype: int64

Podemos observar que a plataforma de `PC` é a mais popular, e entre os primeiros 5 há o clássico `Xbox 360` com mais de 1600 jogos. O `Stadia` é o que menos contando com `5` jogos, e entre os últimos 5 jogos, estão as plataformas mais recentes: `Xbox Series X` com `97` jogos, e `PlayStation 5` com `153` jogos no ranking.

* Quantos jogos há por ano?

In [10]:
metacritic["Release Date"].dt.year.value_counts().reset_index().sort_values(by="index", ascending=False)

,index,Release Date
26,2022,2
6,2021,942
1,2020,1087
3,2019,1014
0,2018,1149
2,2017,1057
4,2016,984
10,2015,864
17,2014,779
18,2013,766


Podemos observar que os jogos que estão no ranking, concentram-se entre `2015` a `2021`.



* Quais são os jogos mais bem avaliados do Video Game WII? <em>(>=75 Metascore)</em>

In [11]:
metacritic[(metacritic["Platform"] == "Wii") & (metacritic["Metascore"] >= 75)].head(15)

,Game,Ranking,Metascore,Platform,Release Date,Type Of Metascore
5,Super Mario Galaxy,6,97,Wii,2007-11-12,Universal Acclaim
6,Super Mario Galaxy 2,7,97,Wii,2010-05-23,Universal Acclaim
57,The Legend of Zelda: Twilight Princess,58,95,Wii,2006-11-19,Universal Acclaim
67,World of Goo,68,94,Wii,2008-10-13,Universal Acclaim
116,Super Smash Bros. Brawl,117,93,Wii,2008-03-09,Universal Acclaim
162,The Legend of Zelda: Skyward Sword,163,93,Wii,2011-11-20,Universal Acclaim
210,Rock Band 2,211,92,Wii,2008-12-18,Universal Acclaim
259,Xenoblade Chronicles,260,92,Wii,2012-04-06,Universal Acclaim
266,Rayman Origins,267,92,Wii,2011-11-15,Universal Acclaim
289,Rock Band 3,290,91,Wii,2010-10-26,Universal Acclaim


* Quais são os jogos antigos mais bem avaliados? (>=75 Metascore e ano mais antigo)

In [12]:
metacritic[ metacritic["Metascore"] >= 75 ].sort_values(by=["Release Date", "Metascore"], ascending=[True, False]).head(15)

,Game,Ranking,Metascore,Platform,Release Date,Type Of Metascore
1445,Full Throttle,1446,86,PC,1995-04-30,Favorable Reviews
656,Duke Nukem 3D,657,89,PC,1996-01-29,Favorable Reviews
88,Sid Meier's Civilization II,89,94,PC,1996-02-29,Universal Acclaim
353,Resident Evil,354,91,PlayStation,1996-03-30,Universal Acclaim
89,Quake,90,94,PC,1996-06-22,Universal Acclaim
4511,Time Commando,4512,80,PC,1996-07-31,Favorable Reviews
611,Tekken 2,612,89,PlayStation,1996-08-25,Favorable Reviews
113,Super Mario 64,114,94,Nintendo 64,1996-09-26,Universal Acclaim
4448,Pilotwings 64,4449,80,Nintendo 64,1996-09-29,Favorable Reviews
178,Wipeout XL,179,93,PlayStation,1996-09-30,Universal Acclaim


Podemos visualizar que a maioria dos jogos são de `PC`, `PlayStation(PS1)` e `Nintendo 64` para a época.

* Quais são os jogos recentes mais bem avaliados?(>=75 Metascore e ano mais recente)

In [13]:
metacritic[ metacritic["Metascore"] >= 75 ].sort_values(by=["Release Date", "Metascore"], ascending=[False, False]).head(15)

,Game,Ranking,Metascore,Platform,Release Date,Type Of Metascore
149,God of War,150,93,PC,2022-01-14,Universal Acclaim
820,Monster Hunter Rise,821,88,PC,2022-01-12,Favorable Reviews
1222,Final Fantasy VII Remake Intergrade,1223,87,PC,2021-12-16,Favorable Reviews
8053,Aeterna Noctis,8054,75,PC,2021-12-15,Favorable Reviews
2096,Shovel Knight Pocket Dungeon,2097,84,PC,2021-12-13,Favorable Reviews
2018,Loop Hero,2019,84,Switch,2021-12-09,Favorable Reviews
5947,Wytchwood,5948,78,PC,2021-12-09,Favorable Reviews
7501,After the Fall,7502,75,PC,2021-12-09,Favorable Reviews
1198,Halo Infinite,1199,87,Xbox Series X,2021-12-08,Favorable Reviews
3456,Halo Infinite,3457,81,PC,2021-12-08,Favorable Reviews


`PC` continua reinando, mas com eles, os jogos da nova geração `Switch`, `Xbox Series X` e `PlayStation 5`

* Quais são os jogos com as piores avalições?(<= 49 Metascore)

In [14]:
pessimo_jogos = metacritic[ metacritic["Metascore"] <= 49 ].sort_values(by="Metascore")
pessimo_jogos.head(15)

,Game,Ranking,Metascore,Platform,Release Date,Type Of Metascore
19105,Family Party: 30 Great Games Obstacle Arcade,19106,11,Wii U,2012-12-04,Overwhelming Dislike
19104,Ride to Hell: Retribution,19105,16,PC,2013-06-24,Overwhelming Dislike
19102,Leisure Suit Larry: Box Office Bust,19103,17,PlayStation 3,2009-05-05,Overwhelming Dislike
19101,Vroom in the Night Sky,19102,17,Switch,2017-04-05,Overwhelming Dislike
19100,Double Dragon II: Wander of the Dragons,19101,17,Xbox 360,2013-04-05,Overwhelming Dislike
19103,Yaris,19104,17,Xbox 360,2007-10-10,Overwhelming Dislike
19099,SPOGS Racing,19100,18,Wii,2008-07-07,Overwhelming Dislike
19098,Ride to Hell: Retribution,19099,19,Xbox 360,2013-06-25,Overwhelming Dislike
19097,Alone in the Dark: Illumination,19098,19,PC,2015-06-11,Overwhelming Dislike
19096,Deal or No Deal,19097,20,DS,2007-07-23,Unfavorable Reviews


In [15]:
pessimo_jogos.shape

(1211, 6)

Podemos ver que são poucos os jogos abaixo de 19 Metascore, mas há muitos que chegam a ser considerados jogos ruins para a crítica. Variando um pouco em alguns anos, e tendo uma variedade de plataformas. Ficando claro que são casos muito específicos. <br>
<em>Fazer um plot analisando por ano os piores jogos</em>

## Fonte

* <sup id="fonte1">1</sup> Na [sessão de rankings](https://www.metacritic.com/browse/games/score/metascore/all/all/filtered) abaixo da paginação mostra que jogos abaixo de 7 reviews críticas são excluídos.

* <sup id="fonte2">2</sup> [O que são essas cores verde, amarelo e vermelho?](https://www.metacritic.com/about-metascores)